In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3060'

In [3]:
torch.cuda.device_count()

1

In [7]:
from torchtext.legacy import data
from torchtext.legacy import datasets

TEXT = data.Field(lower=True, batch_first=True)
train, valid, test = datasets.WikiText2.splits(TEXT,root='data')

downloading wikitext-2-v1.zip


100%|█████████████████████████████████████████████████████████████████████████████| 4.48M/4.48M [00:01<00:00, 2.77MB/s]


extracting


In [8]:
print(len(train[0].text))

2088628


In [9]:
print(train[0].text[:100])

['<eos>', '=', 'valkyria', 'chronicles', 'iii', '=', '<eos>', '<eos>', 'senjō', 'no', 'valkyria', '3', ':', '<unk>', 'chronicles', '(', 'japanese', ':', '戦場のヴァルキュリア3', ',', 'lit', '.', 'valkyria', 'of', 'the', 'battlefield', '3', ')', ',', 'commonly', 'referred', 'to', 'as', 'valkyria', 'chronicles', 'iii', 'outside', 'japan', ',', 'is', 'a', 'tactical', 'role', '@-@', 'playing', 'video', 'game', 'developed', 'by', 'sega', 'and', 'media.vision', 'for', 'the', 'playstation', 'portable', '.', 'released', 'in', 'january', '2011', 'in', 'japan', ',', 'it', 'is', 'the', 'third', 'game', 'in', 'the', 'valkyria', 'series', '.', '<unk>', 'the', 'same', 'fusion', 'of', 'tactical', 'and', 'real', '@-@', 'time', 'gameplay', 'as', 'its', 'predecessors', ',', 'the', 'story', 'runs', 'parallel', 'to', 'the', 'first', 'game', 'and', 'follows', 'the']


In [11]:
train_iter, valid_iter, test_iter = data.BPTTIterator.splits((train, valid, test), batch_size=20, bptt_len=35)

In [21]:
import torch.nn as nn
from torch.autograd import Variable

class RNNModel(nn.Module):
    def __init__(self, ntoken,ninp,nhid,nlayers,dropout=0.5,tie_weights=False):
        #ntoken: 어휘의 단어의 수
        #ninp: LSTM에 입력되는 단어의 임베딩 차원
        #nlayer: LSTM에 사용될 레이어 수
        #Dropout: 과대적합 방지
        #tie_weights: encoder와 decoder에 같은 가중치 사용
        
        super().__init__()
        self.drop = nn.Dropout()
        self.encoder = nn.Embedding(ntoken,ninp)
        self.rnn = nn.LSTM(ninp,nhid,nlayers,dropout=dropout)
        self.decoder = nn.Linear(nhid,ntoken)
        if tie_weights:
            self.decoder.weight = self.encoder.weight
            self.init_weight()
            self.nhid = nhid
            self.nlayers = nlayers
            
        def init_weights(self):
            initrange = 0.1
            self.encoder.weight.data.uniform_(-initrange,initrange)
            self.decoder.bias.data.fill(0)
            self.decoder.weight.data.uniform_(-initrange,initrange)
            
        def forward(self,input,hidden):
            emb = self.drop(self.encoder(input))
            output,hidden = self.rnn(emb,hidden)
            output = self.drop(output)
            s = ouput.size()
            decoded = self.decoder(output.view(s[0]*s[1],s[2]))
            return decoded.view(s[0],s[1],decoded.size(1)),hidden
        
        def init_hidden(self,bsz):
            weight = next(self.parameters()).data
            
            return(Variable(weight.new(self.nlayers,bsz,self.nhid).zero_()), Variable(weight.new(self.nlayers,bsz,self.nhid).zero_()))

In [25]:
def repackage_hidden(h):
    """ 새로운 변수에 hidden 상태를 래핑한다."""
    if type(h) == Variable:
        return Variable(h.data)
    else:
        return tuple(repackage_hidden(v) for v in h)

In [32]:
criterion = nn.CrossEntropyLoss()

def trainf():
    #드롭아웃을 가능하게 하는 트레이닝 모드를 켠다.
    lstm.train()
    total_loss = 0
    start_time = time.time()
    hidden = lstm.init_hidden(batch_size)
    for i,batch in enumerate(train_iter):
        data, targets = batch.text,batch.target.view(-1)
        #각 배치를 시작하고, 숨겨진 상태를 이전에 생성된 상태에서 분리
        #그렇지 않은 경우, 모델은 데이터셋을 시작하기 위해 역전파를 시도
        hidden = repackage_hidden(hidden)
        lstm.zero_grad()
        output, hidden = lstm(data, hidden)
        loss = criterion(output.view(-1, ntokens), targets)
        loss.backward()
        
        #'clip_grad_norm'은 RNNs / LSTM의 기울기 폭발을 방지
        torch.nn.utils.clip_grad_norm(lstm.parameters(), targets)
        for p in lstm.parameters():
            p.data.add_(-lr, p.grad.data)
        
        total_loss += loss.data
        
        if i% log_interval == 0 and i > 0:
            cur_loss =total_loss[0] / log_interval
            elapsed = time.time() - start_time
            (print('|epoch{:3d} | {:5d}/{:5d} batches | lr{:02.2f} | ms/batch {:5.2f} | loss {:5.2f} | ppl {:8.2f}'.format(epoch, i, len(train_iter), lr, elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss))))
            total_loss = 0
            start_time = time.time()   

In [33]:
def evaluate(data_source):
    # 드롭아웃을 비활성화하는 평가 모드를 켠다.
    lstm.eval()
    total_loss = 0
    hidden = lstm.init_hidden(batch_size)
    for batch in data_source:
        data, targets = batch.text, batch.target.view(-1)
        output, hidden = lstm(data, hidden)
        output_flat = output.view(-1, ntokens)
        total_loss += len(data)*criterion(output_flat, targets).data
        hidden = repackage_hidden(hidden)
    return total_loss[0]/(len(data_source.dataset[0].text)//batch_size)

In [34]:
import time

best_val_loss = None
epochs = 40

for epoch in range(1, epochs+1):
    epoch_start_time = time.time()
    trainf()
    val_loss = evaluate(valid_iter)
    print('-'*89)
    print('| end of epoch {:3d} | time: {:5.2f}s | valid loss{:5.2f} | ''valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time), val_loss, math.exp(val_loss)))
    print('-'*89)
    if not best_val_loss or val_loss < vest_val_loss:
        best_val_loss = val_loss
    else:
        # 검증 데이터셍에서 개선이 없을 경우 학습률을 줄임, 20으로 시작
        lr /= 4.0
    

NameError: name 'lstm' is not defined